## imports

In [1]:
!pip install --upgrade pyarrow


In [2]:
from google.colab import drive
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

from tqdm import tqdm
import os

## data

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/quotes/quotes-2020-enhanced10000"
dataset = ds.dataset(path, format="parquet")

create directories

In [4]:
folder_path = "/content/drive/MyDrive/parsed_quotes2/"
folders = ['nationality', 'gender', 'ethnic_group', 'occupation', 'party', 'academic_degree', 'candidacy', 'religion']
none_folders = ['none' + i for i in folders]

for f in folders + none_folders:
  os.mkdir(folder_path + f)


## non none people

In [22]:
path = "/content/drive/MyDrive/parsed_quotes2/author"
path_to_save_none = "/content/drive/MyDrive/parsed_quotes2/none_author"

for i, d in tqdm(enumerate(dataset.to_batches())):
  df = d.to_pandas()
  mask = df['speaker'].apply(lambda x: x == 'None')
  df_autors_none = df[mask.values]
  df_autors_not_none = df[~mask.values]

  #save quotes
  table = pa.Table.from_pandas(df_autors_not_none, preserve_index=True)         
  pq.write_table(table, path + f"/{i}" + ".parquet")

  #save not in description
  table = pa.Table.from_pandas(df_autors_none, preserve_index=True)         
  pq.write_table(table, path_to_save_none + f"/{i}" + ".parquet")


743it [05:25,  2.28it/s]


## split function

In [8]:
def split_and_save(main_path, path_to_save_not_none, path_to_save_none, column_name):
  dataset = ds.dataset(main_path, format="parquet")

  for i, d in tqdm(enumerate(dataset.to_batches())):
    df = d.to_pandas()
    mask = df[column_name].apply(lambda x: str(x) == 'None')
    df_none = df[mask.values]
    df_not_none = df[~mask.values]

    #save not none
    table = pa.Table.from_pandas(df_not_none, preserve_index=True)         
    pq.write_table(table, path_to_save_not_none + f"/{i}" + ".parquet")

    #save none
    table = pa.Table.from_pandas(df_none, preserve_index=True)         
    pq.write_table(table, path_to_save_none + f"/{i}" + ".parquet")

In [ ]:
main_path = "/content/drive/MyDrive/parsed_quotes2/author"
upper_path = "/content/drive/MyDrive/parsed_quotes2/"
for col, none_col in zip(folders, none_folders):
  split_and_save(main_path, upper_path + col, upper_path + none_col, col)

743it [04:46,  2.59it/s]
36it [00:11,  2.90it/s]